<a href="https://colab.research.google.com/github/moraesleonardo/Agrupamentos_para_Machine_Learning/blob/main/Agrupamento_para_Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Agrupamentos

Utilizando dados de sensores

Fonte: Kevin Arvai (adaptado)

## Exemplo 1

In [ ]:
# Instalando a biblioteca Kneed
!pip install kneed

# Importando bibliotecas

import matplotlib.pyplot as plt
import pandas as pd
from kneed import KneeLocator
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

In [ ]:
# Carregando a base de dados de sensores
base = pd.read_excel('Dados_sensores_KMeans.xlsx')

In [ ]:
# Limitando um pouco o tamanho da base para facilitar a interpretação dos dados
base = base.head(20000)
base

In [ ]:
# Separando as variáveis independentes da variável-alvo.
features = base[['sensor1', 'sensor2']]
true_labels = base[['label']]

In [ ]:
# Executando a normalização dos dados para execução do algoritmo
# O StandardScale normaliza aplicando o Z-score [z = (x - u)/s]
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)
scaled_features

In [ ]:
# Visualizando a base de dados
fig, ax1= plt.subplots(figsize=(8, 6))
ax1.scatter(scaled_features[:, 0], scaled_features[:, 1])

In [ ]:
# Instanciando o algoritmo K-Means
kmeans = KMeans(init='random', n_clusters=3, n_init=10, max_iter=300, random_state=42)
# Aplicando o algoritmo
kmeans.fit(scaled_features)

In [ ]:
# Algumas informações sobre o processo de treinamento
# O valor do SSE (Sum of the Squared Error) mais baixo
# O SSE é definido como a soma dos quadrados das distâncias euclidianas de cada ponto ao seu respectivo centróide.
# Por ser uma medida de erro, o objetivo do K-Means é minimizá-la.
kmeans.inertia_ # SSE

In [ ]:
# Coordenadas finais dos centróides
kmeans.cluster_centers_

In [ ]:
# Número de iterações necessárias para alcançar a convergência
kmeans.n_iter_

In [ ]:
# Olhando os grupos estabelecidos pelo algoritmo para os dados iniciais
kmeans.labels_[:5]

In [ ]:
# Visualizando com os grupos definidos
fig, ax1= plt.subplots(figsize=(8, 6))
fte_colors = {0:'red', 1:'green', 2:'blue'}
km_colors = [fte_colors[label] for label in kmeans.labels_]
ax1.scatter(scaled_features[:, 0], scaled_features[:, 1], c=km_colors)

In [ ]:
# E como descobrir o número K?

# Podemos usar o método do cotovelo.
# Calculando SSEs para vários Ks e armazenando os resultados em uma lista
sse = []
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, n_init=10, max_iter=300, random_state=42)
    kmeans.fit(scaled_features)
    sse.append(kmeans.inertia_)


# Visualizando
plt.plot(range(1, 11), sse)
plt.xticks(range(1, 11))
plt.xlabel('Number of Clusters')
plt.ylabel('SSE')

In [ ]:
# Porém, nem sempre a escolha do melhor K é tão óbvia. Vamos usar uma biblioteca para isso.
kl = KneeLocator(range(1, 11), sse, curve='convex', direction='decreasing')
kl.elbow

In [ ]:
# Outra forma de escolha é através de uma métrica chamada Coeficiente Silhouette
'''
The silhouette coefficient is a measure of cluster cohesion and separation. 
It quantifies how well a data point fits into its assigned cluster based on two factors:
- How close the data point is to other points in the cluster
- How far away the data point is from points in other clusters
Silhouette coefficient values range between -1 and 1. 
Larger numbers indicate that samples are closer to their clusters than they are to other clusters.

In the scikit-learn implementation of the silhouette coefficient, the average 
silhouette coefficient of all the samples is summarized into one score. 
The silhouette score() function needs a minimum of two clusters, or it will raise an exception.
'''

# A list holds the silhouette coefficients for each k
silhouette_coefficients = []

# Notice you start at 2 clusters for silhouette coefficient
for k in range(2, 11):
    kmeans = KMeans(n_clusters=k, n_init=10, max_iter=300, random_state=42)
    kmeans.fit(scaled_features)
    score = silhouette_score(scaled_features, kmeans.labels_)
    silhouette_coefficients.append(score)

# Visualizando
plt.plot(range(2, 11), silhouette_coefficients)
plt.xticks(range(2, 11))
plt.xlabel("Number of Clusters")
plt.ylabel("Silhouette Coefficient")

## Exemplo 2

In [ ]:
# Importando as bibliotecas

import pandas as pd
import matplotlib.pyplot as plt
from kneed import KneeLocator
from sklearn.cluster import KMeans, DBSCAN
from sklearn.preprocessing import StandardScaler

In [ ]:
# Carregando a base de dados de sensores
base2 = pd.read_excel('Dados_sensores_KMeans_2.xlsx')

In [ ]:
# Separando as variáveis independentes da variável-alvo.
features = base2[['sensor1', 'sensor2']]
true_labels = base2[['label']]

In [ ]:
# Executando a normalização dos dados para execução do algoritmo
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

In [ ]:
# Visualizando a base de dados
fig, ax1= plt.subplots(figsize=(8, 6))
ax1.scatter(scaled_features[:, 0], scaled_features[:, 1])

In [ ]:
# Escolhendo o melhor K para o K-Means
sse = []
for k in range(1, 11):
    kmeans_kwargs = {'init':'random', 'n_init':10, 'max_iter':300,'random_state':42}
    
    kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
    kmeans.fit(scaled_features)
    sse.append(kmeans.inertia_)

# Visualizando
plt.plot(range(1, 11), sse)
plt.xticks(range(1, 11))
plt.xlabel('Number of Clusters')
plt.ylabel('SSE')

In [ ]:
# Definindo o melhor K com a biblioteca apropriada
kl = KneeLocator(range(1, 11), sse, curve='convex', direction='decreasing')
kl.elbow

In [ ]:
# Instanciando os algoritmos K-Means e DBSCAN
kmeans = KMeans(n_clusters=kl.elbow)
dbscan = DBSCAN(eps=0.3)
'''
Distância: euclidiana

EPS: The maximum distance between two samples for one to be considered as in 
     the neighborhood of the other. This is not a maximum bound on the distances 
     of points within a cluster. 
     This is the most important DBSCAN parameter to choose appropriately for 
     your data set and distance function.

Mais informações sobre o DBSCAN (1996)
<https://scikit-learn.org/stable/modules/clustering.html#dbscan>
Exemplo interessante de aplicação do DBSCAN
<https://www.youtube.com/watch?v=h53WMIImUuc>
'''

In [ ]:
# Aplicando os algoritmos
kmeans.fit(scaled_features)
dbscan.fit(scaled_features)

In [ ]:
# Avaliando os grupos encontrados visualmente
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 6), sharex=True, sharey=True)
fig.suptitle('Comparação de algoritmos de clustering', fontsize=16)
fte_colors = {0:'red', 1:'green', 2:'blue'}
# Visualização do K-Means
km_colors = [fte_colors[label] for label in kmeans.labels_]
ax1.scatter(scaled_features[:, 0], scaled_features[:, 1], c=km_colors)
ax1.set_title('K-Means', fontdict={"fontsize": 12})
# Visualização do DBSCAN
db_colors = [fte_colors[label] for label in dbscan.labels_]
ax2.scatter(scaled_features[:, 0], scaled_features[:, 1], c=db_colors)
ax2.set_title('DBSCAN', fontdict={"fontsize": 12})